In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
cd /content

/content


In [ ]:
pwd

'/content'

In [ ]:
cd /content/drive/MyDrive/

/content/drive/MyDrive


In [ ]:
cd DAAI2022/

/content/drive/MyDrive/DAAI2022


In [ ]:
cd class

/content/drive/MyDrive/DAAI2022/class


In [ ]:
cd code

/content/drive/MyDrive/DAAI2022/class/code


In [ ]:
ls

alexnet.py  data.py  main.py  PACS/  __pycache__/  txt_lists/


In [ ]:
!git clone https://github.com/silvia1993/DANN_Template.git

Cloning into 'DANN_Template'...
remote: Enumerating objects: 23, done.
remote: Total 23 (delta 0), reused 0 (delta 0), pack-reused 23
Unpacking objects: 100% (23/23), done.


In [ ]:
mv DANN_Template code

In [ ]:
!git clone https://github.com/MachineLearning2020/Homework3-PACS

Cloning into 'Homework3-PACS'...
remote: Enumerating objects: 10032, done.
remote: Total 10032 (delta 0), reused 0 (delta 0), pack-reused 10032
Receiving objects: 100% (10032/10032), 174.13 MiB | 16.46 MiB/s, done.
Resolving deltas: 100% (1/1), done.
Checking out files: 100% (9993/9993), done.


In [ ]:
mv Homework3-PACS/PACS code/

In [ ]:
cd code

In [ ]:
ls

alexnet.py  data.py  main.py  PACS/  txt_lists/


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Subset, DataLoader

from torchvision import transforms

from data import Dataset
from alexnet import AlexNet
from torch.utils.model_zoo import load_url as load_state_dict_from_url

DATA_DIR = 'PACS/'

model_urls = {
    'alexnet': 'https://download.pytorch.org/models/alexnet-owt-4df8aa71.pth',
}

DEVICE = 'cuda'
NUM_CLASSES = 7
BATCH_SIZE = 256
LR = 1e-3            # The initial Learning Rate
MOMENTUM = 0.9       # Hyperparameter for SGD, keep this at 0.9 when using SGD
WEIGHT_DECAY = 5e-5  # Regularization, you can keep this at the default
NUM_EPOCHS = 30      # Total number of training epochs (iterations over dataset)
STEP_SIZE = 20       # How many epochs before decreasing learning rate (if using a step-down policy)
GAMMA = 0.1          # Multiplicative factor for learning rate step-down

LOG_FREQUENCY = 5

# Define transforms for training phase
train_transform = transforms.Compose([transforms.Resize(256),
                                      transforms.CenterCrop(224),
                                      transforms.ToTensor(),
                                      transforms.Normalize([0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])
# Define transforms for the evaluation phase
eval_transform = transforms.Compose([transforms.Resize(256),
                                      transforms.CenterCrop(224),
                                      transforms.ToTensor(),
                                      transforms.Normalize([0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])


# Prepare Pytorch train/test Datasets
source_dataset = Dataset(DATA_DIR, dataset='cartoon',  transform=train_transform)
target_dataset = Dataset(DATA_DIR, dataset='sketch', transform=eval_transform)

# Check dataset sizes
print('Source Dataset: {}'.format(len(source_dataset)))
print('Target Dataset: {}'.format(len(target_dataset)))

source_dataloader = DataLoader(source_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2, drop_last=True)
target_dataloader = DataLoader(target_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)


Source Dataset: 2344
Target Dataset: 3929


In [ ]:
net = AlexNet() # Loading AlexNet model
state_dict = load_state_dict_from_url(model_urls['alexnet'], progress=True)
net.load_state_dict(state_dict,strict=False)

net.classifier[6] = nn.Linear(4096, NUM_CLASSES)

criterion = nn.CrossEntropyLoss() # for classification, we use Cross Entropy
parameters_to_optimize = net.parameters() # In this case we optimize over all the parameters of AlexNet
optimizer = optim.SGD(parameters_to_optimize, lr=LR, momentum=MOMENTUM, weight_decay=WEIGHT_DECAY)

scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=STEP_SIZE, gamma=GAMMA)

net = net.to(DEVICE) # this will bring the network to GPU if DEVICE is cuda

current_step = 0

Downloading: "https://download.pytorch.org/models/alexnet-owt-4df8aa71.pth" to /root/.cache/torch/hub/checkpoints/alexnet-owt-4df8aa71.pth


  0%|          | 0.00/233M [00:00<?, ?B/s]

In [ ]:
for epoch in range(NUM_EPOCHS):

    # Iterate over the dataset
    for images, labels in source_dataloader:

      images = images.to(DEVICE)
      labels = labels.to(DEVICE)

      net.train() # Sets module in training mode

      optimizer.zero_grad() # Zero-ing the gradients
      
      outputs = net(images) # Forward pass to the network
      loss = criterion(outputs, labels) # Compute loss based on output and ground truth

      if current_step % LOG_FREQUENCY == 0:
        print('Step {}, Loss {}'.format(current_step, loss.item()))

      loss.backward()  # backward pass: computes gradients
      optimizer.step() # update weights based on accumulated gradients

      current_step += 1

    # Step the scheduler
    scheduler.step()

Step 0, Loss 2.050856828689575
Step 5, Loss 1.4250600337982178
Step 10, Loss 0.7270407676696777
Step 15, Loss 0.6061829924583435
Step 20, Loss 0.49536454677581787
Step 25, Loss 0.617541491985321
Step 30, Loss 0.36338552832603455
Step 35, Loss 0.3451552093029022
Step 40, Loss 0.35067233443260193
Step 45, Loss 0.35142332315444946
Step 50, Loss 0.20637089014053345
Step 55, Loss 0.2442626655101776
Step 60, Loss 0.20628827810287476
Step 65, Loss 0.22314919531345367
Step 70, Loss 0.20487365126609802
Step 75, Loss 0.17195941507816315
Step 80, Loss 0.14387917518615723
Step 85, Loss 0.17844249308109283
Step 90, Loss 0.17223964631557465
Step 95, Loss 0.11981681734323502
Step 100, Loss 0.096489816904068
Step 105, Loss 0.1325274109840393
Step 110, Loss 0.089801125228405
Step 115, Loss 0.06002917140722275
Step 120, Loss 0.07965579628944397
Step 125, Loss 0.1211298257112503
Step 130, Loss 0.07482512295246124
Step 135, Loss 0.04384317994117737
Step 140, Loss 0.08267012983560562
Step 145, Loss 0.07934

In [ ]:
net.train(False) # Set Network to evaluation mode

running_corrects = 0
for images, labels in target_dataloader:

    images = images.to(DEVICE)
    labels = labels.to(DEVICE)

    outputs = net(images)
    _, preds = torch.max(outputs.data, 1)
    running_corrects += torch.sum(preds == labels.data).data.item()

accuracy = running_corrects / float(len(target_dataset))

print('Accuracy on the target domain: {}'.format(accuracy))

Accuracy on the target domain: 0.4858742682616442


In [ ]:
from alexnet import AlexNetDA

pretrained_net = AlexNet() # Loading AlexNet model
state_dict = load_state_dict_from_url(model_urls['alexnet'], progress=True)
pretrained_net.load_state_dict(state_dict,strict=False)
pretrained_net.classifier[6] = nn.Linear(4096, NUM_CLASSES)

netDA = AlexNetDA(pretrained_net)


criterion = nn.CrossEntropyLoss() # for classification, we use Cross Entropy
parameters_to_optimize = netDA.parameters() # In this case we optimize over all the parameters of AlexNet
optimizerDA = optim.SGD(parameters_to_optimize, lr=LR, momentum=MOMENTUM, weight_decay=WEIGHT_DECAY)

schedulerDA = optim.lr_scheduler.StepLR(optimizerDA, step_size=STEP_SIZE, gamma=GAMMA)

netDA = netDA.to(DEVICE) # this will bring the network to GPU if DEVICE is cuda

current_step = 0

In [ ]:
target_DA_loader = DataLoader(target_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2, drop_last=True)

iter_target = iter(target_DA_loader)

alpha = 0.15

for epoch in range(NUM_EPOCHS):

    # Iterate over the dataset
    for images, labels in source_dataloader:

      images = images.to(DEVICE)
      labels = labels.to(DEVICE)

      netDA.train() # Sets module in training mode

      optimizerDA.zero_grad() # Zero-ing the gradients
      
      outputs = netDA(images) # Forward pass to the network
      loss = criterion(outputs, labels) # Compute loss based on output and ground truth

      outputs = netDA(images, DC=True, alpha=alpha)
      source_DA_labels = torch.zeros(len(images), dtype=torch.int64).to(DEVICE)
      loss_source_DA = criterion(outputs, source_DA_labels)

      try: 
        target_images, _ = next(iter_target)
      except StopIteration: 
        iter_target = iter(target_DA_loader)
        target_images, _ = next(iter_target)

      target_images = target_images.to(DEVICE)
      outputs = netDA(target_images, DC=True, alpha=alpha)
      target_DA_labels = torch.ones(len(images), dtype=torch.int64).to(DEVICE)
      loss_target_DA = criterion(outputs, target_DA_labels)

      if current_step % LOG_FREQUENCY == 0:
        print('Step {}, Loss {}, DA Loss {}'.format(current_step, loss.item(), loss_source_DA.item() + loss_target_DA.item()))

      total_loss = loss + loss_source_DA + loss_target_DA
      total_loss.backward()  # backward pass: computes gradients

      optimizerDA.step() # update weights based on accumulated gradients

      current_step += 1

    # Step the scheduler
    schedulerDA.step()

Step 0, Loss 2.422733783721924, DA Loss 1.9086845517158508
Step 5, Loss 1.433384656906128, DA Loss 0.08123700693249702
Step 10, Loss 0.7720676064491272, DA Loss 0.06458252854645252
Step 15, Loss 0.5668811202049255, DA Loss 0.0645255334675312
Step 20, Loss 0.4801631271839142, DA Loss 0.057109685614705086
Step 25, Loss 0.5503724217414856, DA Loss 0.24424083769554272
Step 30, Loss 0.4698273539543152, DA Loss 0.6802689768373966
Step 35, Loss 0.4325312674045563, DA Loss 2.6597299612960796
Step 40, Loss 0.5722215175628662, DA Loss 1.26906356215477
Step 45, Loss 0.4729083776473999, DA Loss 0.9743311852216721
Step 50, Loss 0.4898388385772705, DA Loss 0.7069421112537384
Step 55, Loss 0.3423309624195099, DA Loss 0.8821365684270859
Step 60, Loss 0.29429566860198975, DA Loss 1.5094197392463684
Step 65, Loss 0.33846381306648254, DA Loss 0.5904172360897064
Step 70, Loss 0.38627365231513977, DA Loss 0.7021835744380951
Step 75, Loss 0.24948664009571075, DA Loss 0.39710066467523575
Step 80, Loss 0.3156

In [ ]:
netDA.train(False) # Set Network to evaluation mode

running_corrects = 0
for images, labels in target_dataloader:

    images = images.to(DEVICE)
    labels = labels.to(DEVICE)

    outputs = netDA(images)
    _, preds = torch.max(outputs.data, 1)
    running_corrects += torch.sum(preds == labels.data).data.item()

accuracy = running_corrects / float(len(target_dataset))

print('Accuracy on the target domain: {}'.format(accuracy))

Accuracy on the target domain: 0.5795367778060575
